In [378]:
from keras import backend as K
import tensorflow as tf

import numpy as np
import keras
from keras.preprocessing.text import Tokenizer

nb_discussion = 3
nb_replique_max = 5

#TEXT
text_decoupe = [["The sun is shining in June! It is beautiful.","September is grey.","Life is beautiful in August.",
         "I like it","This and other things?"],
        ["The sun is shining in Fevrier! It is beautiful.","Decembre is grey.","Life is beautiful in Monday.",
         "I like it","This and other things?"],
        ["The sun is shining in Tuesday! It is beautiful.","Juin is grey.","Life is beautiful in August.",
         "I like it","This and other things?"]]
text = ["The sun is shining in June! It is beautiful.","September is grey.","Life is beautiful in August.",
         "I like it","This and other things?", "The sun is shining in Fevrier! It is beautiful.","Decembre is grey.",
         "Life is beautiful in Monday.","I like it","This and other things?",
         "The sun is shining in Tuesday! It is beautiful.","Juin is grey.","Life is beautiful in August.",
         "I like it","This and other things?"]


#TOKENIZER
token = keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                           lower=True, split=" ", char_level=False)
token.fit_on_texts(text)
nombre_mots = len(token.word_counts)

data = np.zeros((nb_discussion, nb_replique_max, nombre_mots))

for i in range(0,nb_discussion):
    data[i,:,:] = token.texts_to_matrix(text_decoupe[i])[:,1:]
    
    
#EMBEDDING
embedding_output_dim = 10

input_text = keras.layers.Input(shape =(nb_replique_max,nombre_mots,), name = "main")
embedding_layer = keras.layers.Embedding(nombre_mots, embedding_output_dim, input_length = None,
                                        embeddings_initializer='uniform',embeddings_regularizer=None, 
                                        activity_regularizer=None,embeddings_constraint=None,
                                        mask_zero=False)(input_text)


#BAG_OF_WORDS
#bag = keras.layers.GlobalAveragePooling1D()(embedding_layer)
bag = keras.layers.Lambda(lambda x: K.sum(x, axis=2), output_shape=(nb_replique_max, embedding_output_dim,))(embedding_layer)


#INPUTBIS
A = np.random.rand(3,5,4)

embedding_incise_dim = 4
input_incise = keras.layers.Input((nb_replique_max, embedding_incise_dim, ), name = "aux")


#MERGING
merged = keras.layers.concatenate([bag, input_incise])


#LSTM
units = 6
lstm = keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                  kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                  unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None,
                  activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None,
                  dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=True, return_state=False,
                  go_backwards=False, stateful=False, unroll=False) (merged)


#MODEL
model = keras.Model(inputs = [input_text, input_incise] , outputs = lstm)

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#print(model.predict([data,A]).shape)


(3, 5, 6)


In [27]:
#ONEHOT_ENCODING
from sklearn.preprocessing import OneHotEncoder
entree = [[2,1,2,1,2],[1,2,3,4,5],[3,6,5,0,0]]

nb_personnages = np.max(entree)
enc = OneHotEncoder(n_values = nb_personnages + 1)

enc.fit(entree)
Y = enc.transform(entree).toarray().reshape(nb_discussion, nb_replique_max, nb_personnages + 1)[:,:,1:]


In [28]:
model.fit([data,A], Y)

Epoch 1/1
3/3 [==============================] - 1s 308ms/step - loss: 2.8737 - acc: 0.4000


In [510]:
donne = []
total = []
discussion = []
personnage = []
personnage_total = []
discussion_int = 1
dictionnaire = dict()
dic_int = 1

with open("corpus/REAL_ALL_CONTENTS_PP.txt","r") as f:
    text = f
    for line in text:
        block = line.split("\t")
        if(int(block[0]) != discussion_int):
            discussion_int += 1
            donne.append(discussion)
            discussion = []
            personnage_total.append(personnage)
            personnage = []
        if(block[1] not in dictionnaire):
            dictionnaire[block[1]] = dic_int
            dic_int += 1
        personnage.append(dictionnaire.get(block[1]))

        discussion.append(block[2])
        total.append(block[2])
    donne.append(discussion)
    personnage_total.append(personnage)
l_max = max([len(disc) for disc in donne])

In [511]:
token = keras.preprocessing.text.Tokenizer(num_words = 100, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                           lower=True, split=" ", char_level=False)
token.fit_on_texts(total)
#longueur = len(token.word_counts)
longueur = 100

data = np.zeros((discussion_int, l_max, longueur))

for i in range(0,discussion_int):
    temp = token.texts_to_matrix(donne[i])#[:,1:]
    if(l_max != len(temp)):
        pad = np.zeros((l_max-len(temp),longueur))
        Mat = np.concatenate((temp,pad), axis = 0)
    data[i,:,:] = Mat
people = []

for pers in personnage_total:
    people.append(np.pad(pers, (0,l_max-len(pers)), 'constant', constant_values=(0, 0)))



In [512]:
nb_personnages = np.max(people)
enc = OneHotEncoder(n_values = nb_personnages + 1)

enc.fit(people)
Y = enc.transform(people).toarray().reshape(discussion_int, l_max, nb_personnages + 1)#[:,:,1:]

embedding_incise_dim = nb_personnages + 1
Z = np.zeros((discussion_int, l_max, embedding_incise_dim))

In [513]:
li = []
data_zero = np.zeros((discussion_int, l_max, longueur))
# res = model.predict([data_zero[0:1], Z[0:1]])
# for i in range(0,l_max):
#     li.append(np.argmax(res[0,i,:]))
# print(li)

#FAUX_EXEMPLE
from random import randint
Y_fake = np.zeros((discussion_int, l_max, nb_personnages+1))
Z_fake = np.zeros((discussion_int, l_max, nb_personnages+1))

for i in range(discussion_int):
    for j in range(l_max):
        r = randint(0,nb_personnages)
        Y_fake[i,j,r] = 1
        Z_fake[i,j,r] = 1

In [516]:
nb_discussion = discussion_int
nb_replique_max = l_max
nombre_mots = longueur
embedding_incise_dim = nb_personnages+1

#EMBEDDING
embedding_output_dim = 1

input_text = keras.layers.Input(shape =(nb_replique_max, nombre_mots,), name = "main")
embedding_layer = keras.layers.Embedding(nombre_mots, embedding_output_dim, input_length = None,
                                        embeddings_initializer='uniform',embeddings_regularizer=None, 
                                        activity_regularizer=None,embeddings_constraint=None,
                                        mask_zero=False)(input_text)


#BAG_OF_WORDS
bag = keras.layers.Lambda(lambda x: K.sum(x, axis=2), output_shape=(nb_replique_max, embedding_output_dim,))(embedding_layer)


#INPUTBIS
input_incise = keras.layers.Input((nb_replique_max, embedding_incise_dim, ), name = "aux")


#MERGING
merged = keras.layers.concatenate([bag, input_incise])


#LSTM
units = nb_personnages + 1
#lstm = keras.layers.LSTM(units, return_sequences=True) (merged)
lstm = keras.layers.LSTM(units, return_sequences=True) (input_incise)

#MODEL

sample = np.array([[int(col != 0) for col in lig] for lig in people])

#OUT
out = keras.layers.Activation('softmax') (lstm)

model = keras.Model(inputs = [input_text, input_incise] , outputs = out)
#model = keras.Model(inputs = input_incise, outputs = out)
sgd = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', 
              metrics=['accuracy'], sample_weight_mode='temporal')


#data_zero = np.zeros((discussion_int, l_max, longueur))
x_train = [data,Z]
y_train = Y
model.fit(x_train, y_train,
          epochs=20,
          batch_size=1,
          sample_weight = sample)

Epoch 1/20
61/61 [==============================] - 6s 96ms/step - loss: 3.5098 - acc: 0.1242
Epoch 2/20
61/61 [==============================] - 2s 30ms/step - loss: 3.4777 - acc: 0.1159
Epoch 3/20
61/61 [==============================] - 2s 30ms/step - loss: 3.4506 - acc: 0.1159
Epoch 4/20
61/61 [==============================] - 2s 30ms/step - loss: 3.4282 - acc: 0.1159
Epoch 5/20
61/61 [==============================] - 2s 30ms/step - loss: 3.4091 - acc: 0.1159
Epoch 6/20
61/61 [==============================] - 2s 30ms/step - loss: 3.3924 - acc: 0.1159
Epoch 7/20
61/61 [==============================] - 2s 30ms/step - loss: 3.3773 - acc: 0.1159
Epoch 8/20
61/61 [==============================] - 2s 30ms/step - loss: 3.3633 - acc: 0.1159
Epoch 9/20
61/61 [==============================] - 2s 30ms/step - loss: 3.3501 - acc: 0.1159
Epoch 10/20
61/61 [==============================] - 2s 31ms/step - loss: 3.3373 - acc: 0.1159
Epoch 11/20
61/61 [==============================] - 2s 32m

In [508]:
Z_fake[:] = 0
Z_fake[:, :, 5] = 1
Z_fake

array([[[0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.]]])

In [509]:
prediction = model.predict(Z_fake)

display(prediction)
display(prediction.argmax(axis=2))

array([[[0.10975358, 0.10491947, 0.10285795, 0.11516447, 0.11314581, 0.1270309 , 0.10674074,
         0.11715734, 0.1032297 ],
        [0.10882805, 0.09977198, 0.09818061, 0.11740898, 0.11424237, 0.13831694, 0.10388377,
         0.12004247, 0.09932473],
        [0.10824458, 0.0957982 , 0.09556642, 0.11858702, 0.11502451, 0.14586535, 0.10205936,
         0.12141627, 0.09743826],
        [0.10790848, 0.09284022, 0.09411175, 0.11914074, 0.11570087, 0.15080579, 0.10089497,
         0.12206629, 0.09653091],
        [0.10773817, 0.09067175, 0.0933149 , 0.11934382, 0.11632665, 0.1540028 , 0.10014976,
         0.12236209, 0.09609   ],
        [0.1076732 , 0.08908736, 0.09289556, 0.11936473, 0.11690675, 0.1560497 , 0.09967295,
         0.12248108, 0.09586864]],

       [[0.10975358, 0.10491947, 0.10285795, 0.11516447, 0.11314581, 0.1270309 , 0.10674074,
         0.11715734, 0.1032297 ],
        [0.10882805, 0.09977198, 0.09818061, 0.11740898, 0.11424237, 0.13831694, 0.10388377,
         0.12004

array([[5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5]], dtype=int64)

In [407]:
data_zero.shape, Z_fake.shape

((2, 6, 12), (2, 6, 5))

In [328]:
print(dictionnaire)

{'Mrs. Bennet': 1, 'Mr. Bennet': 2, 'Elizabeth Bennet': 3, 'Kitty Bennet': 4, 'Lydia Bennet': 5, 'Mr. Bingley': 6, 'Mr. Darcy': 7, 'Jane Bennet': 8, 'Charlotte': 9, 'Mary Bennet': 10, 'young male Lucas': 11, 'Sir William': 12, 'Caroline Bingley': 13, 'Catherine and Lydia Bennet': 14, 'Louisa Hurst': 15, 'Mr. Hurst': 16, 'Mr. Collins': 17, 'NOTANUTTERANCE': 18, 'Mr. Wickham': 19, 'Mr. Denny': 20, 'Mrs. Gardiner': 21, 'Maria': 22, 'Lady Catherine': 23, 'Colonel Fitzwilliam': 24, 'Kitty and Lydia Bennet': 25, 'Mrs. Reynolds': 26, 'Mr. Gardiner': 27, 'Elizabeth Bennet and Mr. Darcy': 28, 'Mrs. Hill': 29, 'The Butler': 30, 'Mrs. Phillips': 31, 'UNSURE': 32, 'Mr. Bennet and Mr. Collins': 33}


In [329]:
bon = 0
for num in range(0,61):
    res = model.predict([data[num:num+1] ,Z[num:num+1] ])
    #print(res[0,0,:])
    for d in range(0,l_max):
        if(sample[num,d]!=0):
            if(np.argmax(Y[num,d,:]) == np.argmax(res[0,d,:])):
                bon += 1
                print(np.argmax(Y[num,d,:]))
print(bon)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
99


In [330]:
pred = []
disc = []
for num in range(0,61):
    res = model.predict([data[num:num+1] ,Z[num:num+1] ])
    #print(res[0,0,:])
    for d in range(0,l_max):
        if(sample[num,d]!=0):
            disc.append(np.argmax(res[0,d,:]))
    pred.append(disc)
    disc = []
print(pred)

[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2], [2, 2, 2, 2, 2, 2, 2,

In [405]:
Z_fake.shape

(2, 6, 5)

In [385]:
x_fake = [data_zero, Z_fake]
model = keras.Model(inputs = [input_text, input_incise] , outputs = lstm)
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              sample_weight_mode='temporal')
# model.fit(x_fake, Y_fake,
#           epochs=20,
#           batch_size=1,
#           sample_weight = np.ones((discussion_int,l_max)))

In [363]:
n = 10
predicted = []
real = []
faux = model.predict([data_zero[n:n+1],Z_fake[n:n+1]])
for i in range(0,5):#l_max):
    print(faux[0,i,:])
    predicted.append(np.argmax(faux[0,i,:]))
    real.append(np.argmax(Y_fake[n,i,:]))
print(predicted)
print(real)

[ 0.         0.         0.7615942 -0.         0.         0.
  0.         0.7615942  0.         0.        -0.7615942  0.
 -0.7615942 -0.7615942 -0.        -0.        -0.7615942 -0.
 -0.        -0.         0.         0.         0.         0.
  0.         0.        -0.7615942  0.         0.        -0.
  0.7615942  0.         0.        -0.       ]
[ 0.         0.         0.9640276 -0.         0.         0.
  0.         0.7615942  0.         0.        -0.7615942  0.
 -0.9640276 -0.9640276 -0.        -0.        -0.7615942 -0.
 -0.        -0.         0.         0.         0.         0.
  0.         0.        -0.9640276  0.         0.        -0.
  0.9640276  0.         0.        -0.       ]
[ 0.         0.         0.9950547 -0.         0.         0.
  0.         0.7615942  0.         0.        -0.7615942  0.
 -0.9950547 -0.9950547 -0.        -0.        -0.7615942 -0.
 -0.        -0.         0.         0.         0.         0.
  0.         0.        -0.9950547  0.         0.        -0.
  0.9950

In [353]:
print(data_zero)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
